In [1]:
from funcs import *

In [6]:
def china_circuit() -> QuantumCircuit:            #https://journals.aps.org/prl/pdf/10.1103/PhysRevLett.131.210603 , nur ein rotated surface code mit 8 ancillas
    qr = QuantumRegister(17, "q")              #q0 - q8: Data qubits,  q9 - q16: Ancilla qubits
    cbit = ClassicalRegister(17,"c")            #8 for initaliziation and 9 for readout
    qc = QuantumCircuit(qr,cbit)

    qc.ry(+np.pi/2, 0), qc.ry(+np.pi/2, 1), qc.ry(+np.pi/2, 7), qc.ry(+np.pi/2, 8)

    qc.h(4)
    qc.t(4)
    ##################
    qc.ry(-np.pi/2, 9)
    qc.ry(-np.pi/2, 2), qc.ry(-np.pi/2, 10), qc.ry(-np.pi/2, 11), qc.ry(-np.pi/2, 12)
    qc.ry(-np.pi/2, 4)
    qc.ry(-np.pi/2, 13), qc.ry(-np.pi/2, 14), qc.ry(-np.pi/2, 15)
    qc.ry(-np.pi/2, 8), qc.ry(-np.pi/2, 16)
    ##################      A
    qc.cz(2,11), qc.cz(1,10), qc.cz(4,14), qc.cz(3,13), qc.cz(5,15), qc.cz(8,16)
    ##################
    qc.ry(-np.pi/2, 1), qc.ry(+np.pi/2, 2)
    qc.ry(-np.pi/2, 3), qc.ry(+np.pi/2, 4), qc.ry(-np.pi/2, 5)
    qc.ry(-np.pi/2, 7), qc.ry(+np.pi/2, 8)
    ##################      B
    qc.cz(1,11), qc.cz(10,4), qc.cz(3,14), qc.cz(13,6), qc.cz(15,8), qc.cz(7,16)
    ##################      C
    qc.cz(0,10), qc.cz(9,1), qc.cz(2,12), qc.cz(11,5), qc.cz(4,15), qc.cz(14,7)
    ##################
    qc.ry(-np.pi/2, 0), qc.ry(+np.pi/2, 1)
    qc.ry(+np.pi/2, 3), qc.ry(-np.pi/2, 4), qc.ry(+np.pi/2, 5)
    qc.ry(-np.pi/2, 6), qc.ry(+np.pi/2, 7)
    ##################      D
    qc.cz(9,0), qc.cz(10,3), qc.cz(12,5), qc.cz(11,4), qc.cz(15,7), qc.cz(14,6)
    ##################
    qc.ry(+np.pi/2, 9), qc.ry(+np.pi/2, 0)
    qc.ry(+np.pi/2, 10), qc.ry(+np.pi/2, 11), qc.ry(+np.pi/2, 12)
    qc.ry(+np.pi/2, 4)
    qc.ry(+np.pi/2, 13), qc.ry(+np.pi/2, 14), qc.ry(+np.pi/2, 15), qc.ry(+np.pi/2, 6)
    qc.ry(+np.pi/2, 16)
    
    #X-Stabilizers
    qc.measure(9,9), qc.measure(11,11), qc.measure(14,14), qc.measure(16,16)

    #Z-Stabilizers
    qc.measure(13,13), qc.measure(10,10), qc.measure(15,15), qc.measure(12,12)

    return qc

def china_readout(qc: QuantumCircuit, had: int, pos: int, shots: int, noise = 0):
    code0 = ['000110101', '110110110', '110110101', '110000000', '000110110', '101101101', '011101101', '011011000', '011011011', '110000011', '000000000', '011101110', '101011011', '101101110', '000000011', '101011000']
    code1 = ['010100111', '010010001', '111111111', '001001010', '111001010', '001111111', '100010010', '111111100', '100100100', '100010001', '001001001', '010010010', '100100111', '111001001', '001111100', '010100100']

    if had%4 == 0:
        for i in range(9):
            qc.measure(i+9*pos, 8-i)
    elif had%4 == 1:
        qc.measure(0+9*pos, 8-6)
        qc.measure(1+9*pos, 8-3)
        qc.measure(2+9*pos, 8-0)
        qc.measure(3+9*pos, 8-7)
        qc.measure(4+9*pos, 8-4)
        qc.measure(5+9*pos, 8-1)
        qc.measure(6+9*pos, 8-8)
        qc.measure(7+9*pos, 8-5)
        qc.measure(8+9*pos, 8-2)
    elif had%4 == 2:
        for i in range(9):
            qc.measure(i+9*pos, i)
    elif had%4 == 3:
        qc.measure(0+9*pos, 8-2)
        qc.measure(1+9*pos, 8-5)
        qc.measure(2+9*pos, 8-8)
        qc.measure(3+9*pos, 8-1)
        qc.measure(4+9*pos, 8-4)
        qc.measure(5+9*pos, 8-7)
        qc.measure(6+9*pos, 8-0)
        qc.measure(7+9*pos, 8-3)
        qc.measure(8+9*pos, 8-6)

    p = noise
    p_error = pauli_error([["X",p/2],["I",1-p],["Z",p/2]])
    p_error_2 = pauli_error([["XI",p/4],["IX",p/4],["II",1-p],["ZI",p/4],["IZ",p/4]])

    noise_model = NoiseModel()
    noise_model.add_all_qubit_quantum_error(p_error, ['x', "z", 'h', "id"])  # Apply to single-qubit gates
    noise_model.add_all_qubit_quantum_error(p_error_2, ['cx'])  # Apply to 2-qubit gates

    sim = AerSimulator()
    job = sim.run(qc, noise_model = noise_model, shots=shots)
    result = job.result()
    counts = result.get_counts()

    print(counts)

    register = list(counts.keys())
    bits = [i[8:] for i in register]
    init = [i[:8] for i in register]
    print("bits:", bits)
    print("Anzahl der versch. Bitstring: ", len(bits))
    hmm = list(counts.values())

    for i in range(len(bits)):
        if init[i].count("1") != 0:
            bits[i] = 2
        for j in code0:
            if j == bits[i]:
                bits[i] = 0
                break
        if bits[i] != 0:
            for j in code1:
                if j == bits[i]:
                    bits[i] = 1
                    break
        if bits[i] != 1 and bits[i] != 0:
            bits[i] = 2

    ones = 0
    zeros = 0
    err = 0

    for i in range(len(bits)):
        if bits[i] == 0:
            zeros += hmm[i]
        if bits[i] == 1:
            ones += hmm[i]
        if bits[i] == 2:
            err += hmm[i]
    
    ones = (ones/shots)
    zeros = (zeros/shots)
    err = (err/shots)

    return zeros, ones, err

qc = china_circuit()

# qec(qc, False, 0)

H_L(qc, 0)

zeros, ones, post = china_readout(qc, had=1, pos=0, shots=10000, noise=0.0)

total = zeros + ones

print("Postselection: ", post)
print("With postselec:")
print("Zeros: ", zeros/total)
print("Ones: ", ones/total)

{'00100100101111101': 1, '01000100101111111': 1, '01100000100010011': 2, '01000110110010001': 4, '01100000010100101': 3, '01100000001111110': 4, '01000100000100111': 3, '00100000111001011': 2, '00100100110100110': 4, '00100010001111110': 3, '01000100011111111': 2, '01100100101001000': 4, '01000010001111100': 4, '00000010001111100': 3, '01100000100010000': 8, '01000100101001010': 5, '00100100000100110': 5, '01000000001001001': 6, '00000100000010010': 10, '00000000100100111': 4, '00000110101111111': 6, '00000100101111100': 12, '01000100000010010': 2, '01000000001001010': 2, '00100110011001000': 10, '01000110000100100': 5, '00100010010010000': 2, '01100100011001000': 4, '00100000111111110': 5, '00100000111111101': 9, '01100010001111110': 3, '01100110101001011': 8, '00000010010010001': 3, '01000000111111111': 6, '00000010010010010': 4, '00000010111111111': 5, '00100110011111110': 10, '00100100110010000': 8, '00000000100010010': 7, '01100110011111101': 7, '00100110101111101': 5, '0100010000

In [2]:
def eth_circuit(n=2) -> QuantumCircuit:              #1st anc = third last qubit = syndrome/parity measurement qubit, 2nd anc = sec last qubit = magic state for S Gate, 3rd anc = last qubit = magic state for T-Gate
    qr = QuantumRegister(9*n+1, "q")
    cbit = ClassicalRegister(13,"c")
    qc = QuantumCircuit(qr,cbit)

    anc = qc.num_qubits - 1

    qc.h(0)
    qc.h(1)
    qc.h(7)
    qc.h(8)

    qc.h(4)
    qc.t(4)
    #qc.ry(np.pi/4, 4)

    #Z0 Z1 Z3 Z4 Stabilizer:
    qc.reset(anc)
    qc.cx(0, anc)
    qc.cx(1, anc)
    qc.cx(3, anc)
    qc.cx(4, anc)
    qc.measure(anc,3)

    #Z4 Z5 Z7 Z8 Stabilizer:
    qc.reset(anc)
    qc.cx(4, anc)
    qc.cx(5, anc)
    qc.cx(7, anc)
    qc.cx(8, anc)
    qc.measure(anc,4)

    #X1 X2 X4 X5 Stabilizer:
    qc.reset(anc)
    qc.h(anc)
    qc.cx(anc, 1)
    qc.cx(anc, 2)
    qc.cx(anc, 4)
    qc.cx(anc, 5)
    qc.h(anc)
    qc.measure(anc,1)

    #X3 X4 X6 X7 Stabilizer:
    qc.reset(anc)
    qc.h(anc)
    qc.cx(anc, 3)
    qc.cx(anc, 4)
    qc.cx(anc, 6)
    qc.cx(anc, 7)
    qc.h(anc)
    qc.measure(anc,2)

    #X0 X1 Stabilizer:
    qc.h(anc)
    qc.cx(anc, 0)
    qc.cx(anc, 1)
    qc.h(anc)
    qc.measure(anc,9)

    #X7 X8 Stabilizer:
    qc.reset(anc)
    qc.h(anc)
    qc.cx(anc, 7)
    qc.cx(anc, 8)
    qc.h(anc)
    qc.measure(anc,12)

    #Z3 Z6 Stabilizer:
    qc.reset(anc)
    qc.cx(3, anc)
    qc.cx(6, anc)
    qc.measure(anc,11)

    #Z2 Z5 Stabilizer:
    qc.reset(anc)
    qc.cx(2, anc)
    qc.cx(5, anc)
    qc.measure(anc,10)

    # with qc.if_test((9,1)):             #0
    #     with qc.if_test((1,0)):    
    #         qc.z(0)

    # with qc.if_test((9,1)):             #1
    #     with qc.if_test((1,1)):
    #         qc.z(1)

    # with qc.if_test((1,1)):             #4
    #     with qc.if_test((2,1)):
    #         qc.z(4)

    # with qc.if_test((2,1)):             #7
    #     with qc.if_test((12,1)):
    #         qc.z(7)

    # with qc.if_test((2,0)):             #8
    #     with qc.if_test((12,1)):
    #         qc.z(8)

    # with qc.if_test((9,0)):             #2 und 5
    #     with qc.if_test((1,1)):        
    #         with qc.if_test((2,0)):    
    #             qc.z(2)

    # with qc.if_test((1,0)):             #3 und 6
    #     with qc.if_test((2,1)):
    #         with qc.if_test((12,0)):
    #             qc.z(3)


    # with qc.if_test((11,1)):             #6
    #     with qc.if_test((3,0)):
    #         qc.x(6)

    # with qc.if_test((11,1)):             #3
    #     with qc.if_test((3,1)):
    #         qc.x(3)

    # with qc.if_test((3,1)):             #4
    #     with qc.if_test((4,1)):
    #         qc.x(4)

    # with qc.if_test((4,0)):             #2
    #     with qc.if_test((10,1)):
    #         qc.x(2)

    # with qc.if_test((4,1)):             #5
    #     with qc.if_test((10,1)):
    #         qc.x(5)
    
    # with qc.if_test((11,0)):             #0 und 1
    #     with qc.if_test((3,1)):
    #         with qc.if_test((4,0)):
    #             qc.x(0)
    
    # with qc.if_test((3,0)):             #7 und 8
    #     with qc.if_test((4,1)):
    #         with qc.if_test((10,0)):
    #             qc.x(7)
    return qc

def eth_readout(qc: QuantumCircuit, had: int, pos: int, shots: int, noise = 0):
    code0 = ['000110101', '110110110', '110110101', '110000000', '000110110', '101101101', '011101101', '011011000', '011011011', '110000011', '000000000', '011101110', '101011011', '101101110', '000000011', '101011000']
    code1 = ['010100111', '010010001', '111111111', '001001010', '111001010', '001111111', '100010010', '111111100', '100100100', '100010001', '001001001', '010010010', '100100111', '111001001', '001111100', '010100100']

    if had%4 == 0:
        for i in range(9):
            qc.measure(i+9*pos, 8-i)
    elif had%4 == 1:
        qc.measure(0+9*pos, 8-6)
        qc.measure(1+9*pos, 8-3)
        qc.measure(2+9*pos, 8-0)
        qc.measure(3+9*pos, 8-7)
        qc.measure(4+9*pos, 8-4)
        qc.measure(5+9*pos, 8-1)
        qc.measure(6+9*pos, 8-8)
        qc.measure(7+9*pos, 8-5)
        qc.measure(8+9*pos, 8-2)
    elif had%4 == 2:
        for i in range(9):
            qc.measure(i+9*pos, i)
    elif had%4 == 3:
        qc.measure(0+9*pos, 8-2)
        qc.measure(1+9*pos, 8-5)
        qc.measure(2+9*pos, 8-8)
        qc.measure(3+9*pos, 8-1)
        qc.measure(4+9*pos, 8-4)
        qc.measure(5+9*pos, 8-7)
        qc.measure(6+9*pos, 8-0)
        qc.measure(7+9*pos, 8-3)
        qc.measure(8+9*pos, 8-6)

    p = noise
    p_error = pauli_error([["X",p/2],["I",1-p],["Z",p/2]])
    p_error_2 = pauli_error([["XI",p/4],["IX",p/4],["II",1-p],["ZI",p/4],["IZ",p/4]])

    noise_model = NoiseModel()
    noise_model.add_all_qubit_quantum_error(p_error, ['x', "z", 'h', "id"])  # Apply to single-qubit gates
    noise_model.add_all_qubit_quantum_error(p_error_2, ['cx'])  # Apply to 2-qubit gates

    sim = AerSimulator()
    job = sim.run(qc, noise_model = noise_model, shots=shots)
    result = job.result()
    counts = result.get_counts()

    print(counts)

    register = list(counts.keys())
    bits = [i[4:] for i in register]
    init = [i[:4] for i in register]
    print("Weight-2 stab: ", init)
    print("bits:", bits)
    print("Anzahl der versch. Bitstring: ", len(bits))
    hmm = list(counts.values())

    for i in range(len(bits)):
        if init[i].count("1") != 0:
            bits[i] = 2
        for j in code0:
            if j == bits[i]:
                bits[i] = 0
                break
        if bits[i] != 0:
            for j in code1:
                if j == bits[i]:
                    bits[i] = 1
                    break
        if bits[i] != 1 and bits[i] != 0:
            bits[i] = 2

    ones = 0
    zeros = 0
    err = 0

    for i in range(len(bits)):
        if bits[i] == 0:
            zeros += hmm[i]
        if bits[i] == 1:
            ones += hmm[i]
        if bits[i] == 2:
            err += hmm[i]
    
    ones = (ones/shots)
    zeros = (zeros/shots)
    err = (err/shots)

    return zeros, ones, err

qc = eth_circuit(n = 1)

anc = qc.num_qubits - 1

qec(qc, False, 0)

H_L(qc, 0)

zeros, ones, post = eth_readout(qc, had = 1, pos = 0, shots = 10000, noise = 0.00)

total = zeros + ones

print("Postselection: ", post)
print("With postselec:")
print("Zeros: ", zeros/total)
print("Ones: ", ones/total)

{'0000001001001': 47, '0001010010010': 156, '0000010100111': 47, '0000010010001': 39, '0000001111111': 43, '0000100010010': 43, '0000100100100': 39, '0000111001010': 39, '0000111111111': 55, '0000010100100': 52, '0001100010010': 175, '0001110000000': 159, '0000001001010': 39, '0000110000011': 282, '0001011101101': 137, '0000000110110': 268, '0001111001010': 155, '0001011011011': 149, '0000101011011': 227, '0001000000011': 146, '0001001001001': 177, '0001110110101': 151, '0000000000011': 251, '0001100100111': 152, '0001010100100': 159, '0001110110110': 134, '0001111111100': 157, '0001101101101': 158, '0001100100100': 159, '0001110000011': 154, '0001101011000': 168, '0001000000000': 148, '0001111111111': 148, '0001101011011': 167, '0001000110110': 148, '0001111001001': 152, '0000101011000': 293, '0001101101110': 151, '0000111001001': 44, '0000011101110': 287, '0000111111100': 48, '0000101101110': 255, '0000011101101': 262, '0001001001010': 152, '0000001111100': 45, '0000110110110': 283, 

In [3]:
#oben bissl sus, brauche 2 Runden stabilizer, wobei 
def eth_circuit(n=2) -> QuantumCircuit:              #1st anc = third last qubit = syndrome/parity measurement qubit, 2nd anc = sec last qubit = magic state for S Gate, 3rd anc = last qubit = magic state for T-Gate
    qr = QuantumRegister(9*n+1, "q")
    cbit = ClassicalRegister(1,"c")
    qc = QuantumCircuit(qr,cbit)

    w4stab = ClassicalRegister(4)
    qc.add_register(w4stab)
    w2stab = ClassicalRegister(4)
    qc.add_register(w2stab)

    anc = qc.num_qubits - 1

    qc.h(0)
    qc.h(1)
    qc.h(7)
    qc.h(8)

    qc.h(4)
    qc.t(4)
    #qc.ry(np.pi/4, 4)

    #Z0 Z1 Z3 Z4 Stabilizer:
    qc.reset(anc)
    qc.cx(0, anc)
    qc.cx(1, anc)
    qc.cx(3, anc)
    qc.cx(4, anc)
    qc.measure(anc,w4stab[2])

    #Z4 Z5 Z7 Z8 Stabilizer:
    qc.reset(anc)
    qc.cx(4, anc)
    qc.cx(5, anc)
    qc.cx(7, anc)
    qc.cx(8, anc)
    qc.measure(anc,w4stab[3])

    #X1 X2 X4 X5 Stabilizer:
    qc.reset(anc)
    qc.h(anc)
    qc.cx(anc, 1)
    qc.cx(anc, 2)
    qc.cx(anc, 4)
    qc.cx(anc, 5)
    qc.h(anc)
    qc.measure(anc,w4stab[0])

    #X3 X4 X6 X7 Stabilizer:
    qc.reset(anc)
    qc.h(anc)
    qc.cx(anc, 3)
    qc.cx(anc, 4)
    qc.cx(anc, 6)
    qc.cx(anc, 7)
    qc.h(anc)
    qc.measure(anc,w4stab[1])

    #X0 X1 Stabilizer:
    qc.h(anc)
    qc.cx(anc, 0)
    qc.cx(anc, 1)
    qc.h(anc)
    qc.measure(anc,w2stab[0])

    #X7 X8 Stabilizer:
    qc.reset(anc)
    qc.h(anc)
    qc.cx(anc, 7)
    qc.cx(anc, 8)
    qc.h(anc)
    qc.measure(anc,w2stab[3])

    #Z3 Z6 Stabilizer:
    qc.reset(anc)
    qc.cx(3, anc)
    qc.cx(6, anc)
    qc.measure(anc,w2stab[2])

    #Z2 Z5 Stabilizer:
    qc.reset(anc)
    qc.cx(2, anc)
    qc.cx(5, anc)
    qc.measure(anc,w2stab[1])

    with qc.if_test((w2stab[0],1)):             #0
        with qc.if_test((w4stab[0],0)):    
            qc.z(0)

    with qc.if_test((w2stab[0],1)):             #1
        with qc.if_test((w4stab[0],1)):
            qc.z(1)

    with qc.if_test((w4stab[0],1)):             #4
        with qc.if_test((w4stab[1],1)):
            qc.z(4)

    with qc.if_test((w4stab[1],1)):             #7
        with qc.if_test((w2stab[3],1)):
            qc.z(7)

    with qc.if_test((w4stab[1],0)):             #8
        with qc.if_test((w2stab[3],1)):
            qc.z(8)

    with qc.if_test((w2stab[0],0)):             #2 und 5
        with qc.if_test((w4stab[0],1)):        
            with qc.if_test((w4stab[1],0)):    
                qc.z(2)

    with qc.if_test((w4stab[0],0)):             #3 und 6
        with qc.if_test((w4stab[1],1)):
            with qc.if_test((w2stab[3],0)):
                qc.z(3)


    with qc.if_test((w2stab[2],1)):             #6
        with qc.if_test((w4stab[2],0)):
            qc.x(6)

    with qc.if_test((w2stab[2],1)):             #3
        with qc.if_test((w4stab[2],1)):
            qc.x(3)

    with qc.if_test((w4stab[2],1)):             #4
        with qc.if_test((w4stab[3],1)):
            qc.x(4)

    with qc.if_test((w4stab[3],0)):             #2
        with qc.if_test((w2stab[1],1)):
            qc.x(2)

    with qc.if_test((w4stab[3],1)):             #5
        with qc.if_test((w2stab[1],1)):
            qc.x(5)
    
    with qc.if_test((w2stab[2],0)):             #0 und 1
        with qc.if_test((w4stab[2],1)):
            with qc.if_test((w4stab[3],0)):
                qc.x(0)
    
    with qc.if_test((w4stab[2],0)):             #7 und 8
        with qc.if_test((w4stab[3],1)):
            with qc.if_test((w2stab[1],0)):
                qc.x(7)
    return qc

def eth_readout(qc: QuantumCircuit, had: int, pos: int, shots: int, noise = 0):
    code0 = ['000110101', '110110110', '110110101', '110000000', '000110110', '101101101', '011101101', '011011000', '011011011', '110000011', '000000000', '011101110', '101011011', '101101110', '000000011', '101011000']
    code1 = ['010100111', '010010001', '111111111', '001001010', '111001010', '001111111', '100010010', '111111100', '100100100', '100010001', '001001001', '010010010', '100100111', '111001001', '001111100', '010100100']

    read = ClassicalRegister(9)
    qc.add_register(read)

    if had%4 == 0:
        for i in range(9):
            qc.measure(i+9*pos, read[8-i])
    elif had%4 == 1:
        qc.measure(0+9*pos, read[8-6])
        qc.measure(1+9*pos, read[8-3])
        qc.measure(2+9*pos, read[8-0])
        qc.measure(3+9*pos, read[8-7])
        qc.measure(4+9*pos, read[8-4])
        qc.measure(5+9*pos, read[8-1])
        qc.measure(6+9*pos, read[8-8])
        qc.measure(7+9*pos, read[8-5])
        qc.measure(8+9*pos, read[8-2])
    elif had%4 == 2:
        for i in range(9):
            qc.measure(i+9*pos, i)
    elif had%4 == 3:
        qc.measure(0+9*pos, read[8-2])
        qc.measure(1+9*pos, read[8-5])
        qc.measure(2+9*pos, read[8-8])
        qc.measure(3+9*pos, read[8-1])
        qc.measure(4+9*pos, read[8-4])
        qc.measure(5+9*pos, read[8-7])
        qc.measure(6+9*pos, read[8-0])
        qc.measure(7+9*pos, read[8-3])
        qc.measure(8+9*pos, read[8-6])

    p = noise
    p_error = pauli_error([["X",p/2],["I",1-p],["Z",p/2]])
    p_error_2 = pauli_error([["XI",p/4],["IX",p/4],["II",1-p],["ZI",p/4],["IZ",p/4]])

    noise_model = NoiseModel()
    noise_model.add_all_qubit_quantum_error(p_error, ['x', "z", 'h', "id"])  # Apply to single-qubit gates
    noise_model.add_all_qubit_quantum_error(p_error_2, ['cx'])  # Apply to 2-qubit gates

    sim = AerSimulator()
    job = sim.run(qc, noise_model = noise_model, shots=shots)
    result = job.result()
    counts = result.get_counts()

    print(counts)

    register = list(counts.keys())
    register = [i.replace(" ","") for i in register]
    bits = [i[:9] for i in register]
    init = [i[1:5] for i in register]
    print("Weight-2 stab: ", init)
    print("bits:", bits)
    print("Anzahl der versch. Bitstring: ", len(bits))
    hmm = list(counts.values())

    for i in range(len(bits)):
        if init[i].count("1") != 0:
            bits[i] = 2
        for j in code0:
            if j == bits[i]:
                bits[i] = 0
                break
        if bits[i] != 0:
            for j in code1:
                if j == bits[i]:
                    bits[i] = 1
                    break
        if bits[i] != 1 and bits[i] != 0:
            bits[i] = 2

    ones = 0
    zeros = 0
    err = 0

    for i in range(len(bits)):
        if bits[i] == 0:
            zeros += hmm[i]
        if bits[i] == 1:
            ones += hmm[i]
        if bits[i] == 2:
            err += hmm[i]
    
    ones = (ones/shots)
    zeros = (zeros/shots)
    err = (err/shots)

    return zeros, ones, err

qc = eth_circuit(n = 1)

anc = qc.num_qubits - 1

H_L(qc, 0)

zeros, ones, post = eth_readout(qc, had = 1, pos = 0, shots = 10000, noise = 0.00)

#total = zeros + ones

print("Postselection: ", post)
print("With postselec:")
print("Zeros: ", zeros)
print("Ones: ", ones)
# print("Zeros: ", zeros/total)
# print("Ones: ", ones/total)

{'010100111 0000 0001 0': 1, '111001001 0000 0001 0': 1, '011111011 0001 1011 0': 3, '100100100 0000 0100 0': 4, '010100011 0001 0010 0': 3, '111001010 0000 1101 0': 4, '100010010 0000 0100 0': 3, '100010010 0000 1000 0': 3, '001111111 0000 0101 0': 2, '010010110 0001 1010 0': 4, '011001101 0001 1111 0': 1, '001001001 0000 1001 0': 8, '010100111 0000 1001 0': 5, '010010001 0000 0001 0': 5, '010010010 0000 0001 0': 12, '111001101 0001 0110 0': 4, '000010110 0001 1111 0': 4, '100100000 0001 1110 0': 6, '001111011 0001 0110 0': 6, '010100011 0001 0110 0': 7, '010100000 0001 0110 0': 6, '100010010 0000 0000 0': 3, '001001101 0001 0010 0': 2, '010010010 0000 1101 0': 5, '110010101 0001 1011 0': 3, '111111100 0000 1001 0': 3, '010010001 0000 0101 0': 5, '011111011 0001 0111 0': 6, '111111111 0000 0001 0': 5, '111111111 0000 1001 0': 2, '010100100 0000 0000 0': 4, '010100011 0001 1110 0': 4, '001001001 0000 1000 0': 7, '110100000 0001 1111 0': 5, '001001110 0001 1010 0': 6, '100100111 0000 10